In [115]:
from functools import partial
import subprocess
import glob
import time
import numpy as np

TAB_SIZE  = 15
TIMEOUT   = 40


kiter_executable = "./Release/bin/kiter"
cmdKperiodic = lambda filename : [kiter_executable, "-f", filename,  "-a", "KPeriodicThroughput"]


class BenchmarkResult :
    status_code    = "EMPTY"
    execution_time = None
    stdout         = None

    def __init__ (self, et = None, stdout = None, status_code = "EMPTY") :
        self.status_code = status_code
        self.execution_time = et
        if (type(stdout) == bytes) :
            self.stdout         = stdout.decode("utf-8") 
        else :
            self.stdout         = stdout
            
    def getExecutionTime(self) :
        return self.execution_time
    
    def getComputeTime (self):
        if (not self.stdout) : return None
        for l in self.stdout.split("\n") :
            if "Time" in l :
                return float(l.split(" ")[-1])
    
def runBenchmarkOnce (filename) :
    
    command = cmdKperiodic(filename = filename)
    
    try :
        start_time = time.time()
        result = subprocess.run(command, stdout=subprocess.PIPE,timeout = TIMEOUT)
        end_time = time.time()
        executiontime = (end_time  - start_time) * 1000
        output = result.stdout
        return BenchmarkResult (executiontime  ,  output , "FINISHED")
    except subprocess.TimeoutExpired as e:
        return BenchmarkResult (None  ,  None , "TIMEOUT")
    except FileNotFoundError :
        return BenchmarkResult (None  ,  None , "NOT FOUND")
    except _ as e :
        return BenchmarkResult (None  ,  None , "ERROR")

def runBenchmark (filename, it = 10) :
    execution_times = []
    compute_times = []
    for _ in range(it) :
        res = runBenchmarkOnce (filename)
        execution_times += [res.getExecutionTime()]
        compute_times += [res.getComputeTime()]
    if None in execution_times or None in compute_times :
        return None, None
    return np.median(execution_times) ,  np.median(compute_times)

def formatTime (et , rj = 10) : 
    return '{:.2f} ms'.format(et).rjust(rj) if et else '-'.rjust(rj)
    
def runBenchmarks (filelist) :
    colSize = max([len(x) for x in filelist])

    print("Name".rjust(colSize), "ExecutionTime".rjust(15), "ComputeTime".rjust(15), "Status".rjust(TAB_SIZE))
    for filename in sorted(filelist) :
        execution_time,compute_time  = runBenchmark(filename)
        print(filename.rjust(colSize),
              formatTime(execution_time,TAB_SIZE),
              formatTime(compute_time,TAB_SIZE))


benchmarks  = glob.glob('./benchmark/*.xml')
benchmarksS = glob.glob('./benchmark_sized/*.xml')
benchmarksG = glob.glob('./AGB5CSDF/*.xml')

In [117]:
runBenchmarks (benchmarks)

                                    Name   ExecutionTime     ComputeTime          Status
                      ./benchmark/21.xml         6.68 ms         0.10 ms
            ./benchmark/BlackScholes.xml         8.15 ms         0.42 ms
                    ./benchmark/Echo.xml         9.80 ms         0.32 ms
                    ./benchmark/H264.xml       306.40 ms        32.94 ms
                ./benchmark/JPEG2000.xml        55.19 ms         5.00 ms
                ./benchmark/Pdectect.xml        32.91 ms        10.07 ms
./benchmark/expansion_paper_norm_sdf.xml         6.69 ms         0.10 ms
     ./benchmark/expansion_paper_sdf.xml         9.25 ms         0.14 ms
             ./benchmark/init_sample.xml         8.29 ms         0.15 ms
           ./benchmark/new_benchmark.xml         7.05 ms         0.17 ms
                  ./benchmark/sample.xml         7.58 ms         0.13 ms
       ./benchmark/simpler_benchmark.xml         8.03 ms         0.08 ms


In [118]:
runBenchmarks (benchmarksS)

                               Name   ExecutionTime     ComputeTime          Status
./benchmark_sized/Black-scholes.xml        12.97 ms         0.86 ms
         ./benchmark_sized/Echo.xml      7883.42 ms      7866.23 ms
         ./benchmark_sized/H264.xml       890.17 ms        69.79 ms
     ./benchmark_sized/JPEG2000.xml       682.08 ms       594.25 ms
     ./benchmark_sized/Pdectect.xml        43.94 ms        15.40 ms


In [119]:
runBenchmarks (benchmarksG)

                   Name   ExecutionTime     ComputeTime          Status
./AGB5CSDF/autogen1.xml               -               -
./AGB5CSDF/autogen2.xml               -               -
./AGB5CSDF/autogen3.xml               -               -
./AGB5CSDF/autogen4.xml      1604.75 ms       155.75 ms
./AGB5CSDF/autogen5.xml      9922.34 ms      7661.83 ms
